In [39]:
!pip install scprep phate  anndata umap-learn magic-impute hdbscan
import scprep

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

# Matplotlib command for Jupyter notebooks only
%matplotlib inline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.2 MB 5.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for hdbscan: filename=hdbscan-0.8.28-cp37-cp37m-linux_x86_64.whl size=2342150 sha256=6a1fe30abb9b60f08fcacb0f0ef33e882b9d4d8d37b0acc58eeef7427ca4f119
  Stored in directory: /root/.cache/pip/wheels/6e/7a/5e/259ccc841c085fc41b99ef4a71e896b62f5161f2bc8a14c97a
Successfully built hdbscan


In [2]:

import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform
import umap

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
raw = pd.read_csv('/content/drive/MyDrive/MCF7 PRO/MAGIC/X_magic_impu.csv',sep=',')

In [29]:
raw= raw.set_index('Unnamed: 0')

In [30]:
pca = PCA(n_components=3, random_state=1)

In [31]:
pca_data= pca.fit_transform(raw)

In [32]:
pca_data

array([[ 4.37755278e+00, -9.56144388e-01, -4.27901404e-01],
       [ 6.33660833e-01, -1.72781878e-01,  5.72534188e-01],
       [ 4.57223873e+00, -7.73941281e-01, -8.35028628e-04],
       ...,
       [-3.96239452e+00,  7.33765652e-02, -3.88407227e-01],
       [-2.69363093e+00,  2.27650549e-01, -5.98279277e-02],
       [ 2.88538679e+00,  2.27912211e+00, -3.35205805e-01]])

In [33]:

pca_data_3D= pd.DataFrame(pca_data, columns=['PC1', 'PC2','PC3'])

In [68]:
import plotly.express as px
import plotly

fig = px.scatter_3d(pca_data_3D,  x="PC1", y="PC2",z='PC3', color = raw['MT-ND1'])
             
fig.update_traces(marker=dict(size=3))
fig.update_layout(
                  scene = dict(xaxis=dict(backgroundcolor='white',
                                          color='black',
                                          gridcolor='#E5E5E5'),
                               yaxis=dict(backgroundcolor='white',
                                          color='black',
                                          gridcolor='#E5E5E5'
                                          ),
                               zaxis=dict(backgroundcolor='white',
                                          color='black', 
                                          gridcolor='#E5E5E5')))


fig.show()

**HDBSCAN**

In [58]:
import hdbscan
#clusterer = hdbscan.HDBSCAN(min_cluster_size=10,cluster_selection_epsilon=0.9).fit(scores_df)
import numpy as np
np.random.seed(400)

clusterer = hdbscan.HDBSCAN().fit(pca_data_3D)


In [59]:
clusters = pd.Categorical(clusterer.labels_)

In [60]:
clusters

[2, 4, 2, 2, 2, ..., -1, 4, 4, 4, 0]
Length: 363
Categories (6, int64): [-1, 0, 1, 2, 3, 4]

In [61]:
import plotly.express as px
import plotly

fig = px.scatter_3d(pca_data_3D,  x="PC1", y="PC2",z='PC3', color = clusters)
             
fig.update_traces(marker=dict(size=3))
fig.update_layout(
                  scene = dict(xaxis=dict(backgroundcolor='white',
                                          color='black',
                                          gridcolor='#E5E5E5'),
                               yaxis=dict(backgroundcolor='white',
                                          color='black',
                                          gridcolor='#E5E5E5'
                                          ),
                               zaxis=dict(backgroundcolor='white',
                                          color='black', 
                                          gridcolor='#E5E5E5')))
fig.update_layout(scene_aspectmode='cube')
camera = dict(
    eye=dict(x=-2.5, y=-1., z=0.2),
    
)

fig.update_layout(scene_camera=camera)

fig.show()

In [66]:
cluster = pd.DataFrame(clusters, columns=['Cluster'])

In [67]:
cluster.to_csv('3DPCAHDBSCANIMPUTATIONMAGIC.csv')